In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import sys
sys.argv = sys.argv[:1]
from model import ParserModel, main, Config

import time

from itertools import islice
from sys import stdout
from tempfile import NamedTemporaryFile
import tensorflow as tf
from utils.model import Model
from data import load_and_preprocess_data
from data import score_arcs
from initialization import xavier_weight_init
from parser import minibatch_parse
from utils.generic_utils import Progbar

from tensorflow.python.tools.freeze_graph import freeze_graph
import tfcoreml

In [ ]:
# class Config(object):
#     """Holds model hyperparams and data information.

#     The config class is used to store various hyperparameters and dataset
#     information parameters. Model objects are passed a Config() object at
#     instantiation.
#     """
#     n_word_ids = None # inferred
#     n_tag_ids = None # inferred
#     n_deprel_ids = None # inferred
#     n_word_features = None # inferred
#     n_tag_features = None # inferred
#     n_deprel_features = None # inferred
#     n_classes = None # inferred
#     dropout = 0.5
#     embed_size = None # inferred
#     hidden_size = 4
#     batch_size = 2048
#     n_epochs = 1
#     lr = 0.001
#     l2_beta = 10e-8
#     l2_loss = 0


In [2]:
'''Main function

Args:
debug :
    whether to use a fraction of the data. Make sure to set to False
    when you're ready to train your model for real!
'''
print(80 * "=")
print("INITIALIZING")
print(80 * "=")
config = Config()
data = load_and_preprocess_data(
    max_batch_size=config.batch_size)
transducer, word_embeddings, train_data = data[:3]
dev_sents, dev_arcs = data[3:5]
test_sents, test_arcs = data[5:]
config.n_word_ids = len(transducer.id2word) + 1 # plus null
config.n_tag_ids = len(transducer.id2tag) + 1
config.n_deprel_ids = len(transducer.id2deprel) + 1
config.embed_size = word_embeddings.shape[1]
for (word_batch, tag_batch, deprel_batch), td_batch in \
        train_data.get_iterator(shuffled=False):
    config.n_word_features = word_batch.shape[-1]
    config.n_tag_features = tag_batch.shape[-1]
    config.n_deprel_features = deprel_batch.shape[-1]
    config.n_classes = td_batch.shape[-1]
    break
print(
    'Word feat size: {}, tag feat size: {}, deprel feat size: {}, '
    'classes size: {}'.format(
        config.n_word_features, config.n_tag_features,
        config.n_deprel_features, config.n_classes))

INITIALIZING
Loading word embeddings...['!', '#', '$', '%', '&', "'", "''", "'d", "'ll", "'m", "'re", "'s", "'ve", ',', '-', '--', '-LCB-', '-LRB-', '-RCB-', '-RRB-', '.', '...', '0.2', '1', '1,000', '1.1', '1.2', '1.25', '1.3', '1.4', '1.5', '1.6', '1.7', '1.8', '1.9', '1/2', '1/4', '1/8', '10', '10,000', '10-year', '100', '100,000', '101', '102', '11', '11/16', '110', '12', '12.5', '120', '125', '13', '14', '140', '15', '15,000', '150', '16', '17', '18', '180', '19', '1970s', '1972', '1973', '1974', '1979', '1980', '1980s', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1990s', '1991', '1992', '1993', '1994', '1995', '1999', '2', '2,000', '2.2', '2.3', '2.4', '2.5', '2.6', '2.7', '2.8', '20', '20,000', '200', '200,000', '2009', '21', '22', '225', '23', '24', '25', '250', '250,000', '26', '27', '28', '29', '3', '3,000', '3.1', '3.3', '3.5', '3.6', '3.7', '3.8', '3.9', '3/4', '3/8', '30', '30-year', '300', '31', '32', '33', '34', '35', '350', '36', '37

Determining POS tags...['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
there are 17 tags.
Determining deprel labels...['acl', 'acl:relcl', 'advcl', 'advmod', 'amod', 'appos', 'aux', 'auxpass', 'case', 'cc', 'cc:preconj', 'ccomp', 'compound', 'compound:prt', 'conj', 'cop', 'csubj', 'csubjpass', 'dep', 'det', 'det:predet', 'discourse', 'dobj', 'expl', 'iobj', 'mark', 'mwe', 'neg', 'nmod', 'nmod:npmod', 'nmod:poss', 'nmod:tmod', 'nsubj', 'nsubjpass', 'nummod', 'parataxis', 'punct', 'root', 'xcomp']
there are 39 deprel labels.
Getting training data...

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/tf/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-f39a3b355532>", line 13, in <module>
    max_batch_size=config.batch_size)
  File "/Users/aryavohra/newprojects/spacy-ios/neural-dependency-parser/data.py", line 585, in load_and_preprocess_data
    transition_cache=transition_cache, seed=seed,
  File "/Users/aryavohra/newprojects/spacy-ios/neural-dependency-parser/data.py", line 415, in __init__
    self._construct_all_data()
  File "/Users/aryavohra/newprojects/spacy-ios/neural-dependency-parser/data.py", line 427, in _construct_all_data
    for feat_vecs, td_vec in data_iter:
  File "/Users/aryavohra/newprojects/spacy-ios/neural-dependency-parser/data.py", line 278, in graphs2feats_and_tds
    feat_tups.append(self.pp2feat(pp))
  File "/Users/aryavohra/newprojects/spacy-ios/neural-dependency-parser/d

KeyboardInterrupt: 

# DON'T RERUN THINGS BEFORE THIS

In [ ]:
print(transducer.id2word)
debug = False

In [ ]:
if debug:
    dev_sents = dev_sents[:500]
    dev_arcs = dev_arcs[:500]
    test_sents = test_sents[:500]
    test_arcs = test_arcs[:500]
if not debug:
    weight_file = NamedTemporaryFile(suffix='.weights')
    # weight_file = open("something.weights", mode=)

session = tf.Session()
print("Building model...", end=' ')
start = time.time()
model = ParserModel(transducer, session, config, word_embeddings, is_training=False)
print("took {:.2f} seconds\n".format(time.time() - start))
init = tf.global_variables_initializer()
session.run(init)
output_names = 'output/td_vec'
saver = None if debug else tf.train.Saver()
saver.restore(session, "checkpoints/model.ckpt")

# frozen_graph = tf.compat.v1.graph_util.convert_variables_to_constants(
#     sess=session,
#     input_graph_def=tf.compat.v1.get_default_graph().as_graph_def(),
#     output_node_names=[output_names])
# frozen_graph = tf.compat.v1.graph_util.extract_sub_graph(
#     graph_def=frozen_graph,
#     dest_nodes=[output_names])
# with open('checkpoints/frozen_graph.pb', 'wb') as fout:
#     fout.write(frozen_graph.SerializeToString())
# print(80 * "=")
# print("TRAINING")
# print(80 * "=")
# best_las = 0.

#     for epoch in range(config.n_epochs):
#         print('Epoch {}'.format(epoch))

#         if debug:
#             model.fit_epoch(list(islice(train_data,3)), config.batch_size)
#         else:
#             model.fit_epoch(train_data)
#         stdout.flush()
#         dev_las, dev_uas = model.eval(dev_sents, dev_arcs)
#         best = dev_las > best_las
#         if best:
#             best_las = dev_las
#             if not debug:
#                 saver.save(session, "checkpoints/model.ckpt")
#                 tf.io.write_graph(session.graph_def, './checkpoints/', 'model.pbtxt')
#                 frozen_graph = tf.compat.v1.graph_util.convert_variables_to_constants(
#                     sess=session,
#                     input_graph_def=tf.compat.v1.get_default_graph().as_graph_def(),
#                     output_node_names=[output_names])
#                 frozen_graph = tf.compat.v1.graph_util.extract_sub_graph(
#                     graph_def=frozen_graph,
#                     dest_nodes=[output_names])
#                 with open('checkpoints/frozen_graph.pb', 'wb') as fout:
#                     fout.write(frozen_graph.SerializeToString())

#         print('Validation LAS: ', end='')
#         print('{:.2f}{}'.format(dev_las, ' (BEST!), ' if best else ', '))
#         print('Validation UAS: ', end='')
#         print('{:.2f}'.format(dev_uas))

# if not debug:
#     print()
#     print(80 * "=")
#     print("TESTING")
#     print(80 * "=")
#     print("Restoring the best model weights found on the dev set")
#     saver.restore(session, "checkpoints/model.ckpt")
#     stdout.flush()
#     las,uas = model.eval(test_sents, test_arcs)
#     if las:
#         print("Test LAS: ", end='')
#         print('{:.2f}'.format(las), end=', ')
#     print("Test UAS: ", end='')
#     print('{:.2f}'.format(uas))
#     print("Done!")

In [ ]:
# with tf.Graph().as_default(), tf.Session() as session:
#     print("Building model...", end=' ')
#     start = time.time()
#     model = ParserModel(transducer, session, config, word_embeddings, is_training=True)
#     print("took {:.2f} seconds\n".format(time.time() - start))
#     init = tf.global_variables_initializer()
#     session.run(init)
#     output_names = 'output/td_vec'
#     saver = None if debug else tf.train.Saver()
# #     saver.restore(session, "checkpoints/model.ckpt")
#     frozen_graph = tf.compat.v1.graph_util.convert_variables_to_constants(
#         sess=session,
#         input_graph_def=tf.compat.v1.get_default_graph().as_graph_def(),
#         output_node_names=[output_names])
#     frozen_graph = tf.compat.v1.graph_util.extract_sub_graph(
#         graph_def=frozen_graph,
#         dest_nodes=[output_names])
#     with open('checkpoints/frozen_graph.pb', 'wb') as fout:
#         fout.write(frozen_graph.SerializeToString())
#     saver.restore(session, "checkpoints/model.ckpt")
#     print(test_sents)
#     las,uas = model.eval(test_sents, test_arcs)


In [ ]:
# transducer, word_embeddings, train_data = data[:3]
# dev_sents, dev_arcs = data[3:5]
# test_sents, test_arcs = data[5:]
# print(dev_sents[:1])
# print(dev_arcs[:1])
# import json
# list(islice(train_data,1))[0][0]

gameplan: run minibatch_parse on single sentence pair, extract the feed. understand structure of said feed. try feeding feed into model directly. if it works => mad profit

In [2]:
import importlib
import parser
importlib.reload(parser)

<module 'parser' from '/Users/khushjammu/GitHub/spacy-ios/neural-dependency-parser/parser.py'>

In [ ]:
parser.minibatch_parse(
    [[("What", "PRON"), ("if", "SCONJ"), ("Google", "PROPN"), ("Morphed", "VERB"), ("Into", "ADP"), ("GoogleOS", "PROPN"), ("?", "PUNCT")]],
    model,
    1
)

gameplan: run `load_and_preprocess_data` to create `TrainingIterable`-s, which calls `graphs2feats_and_tds` in its constructor. 

In [3]:
import data
import importlib
importlib.reload(data)

<module 'data' from '/Users/khushjammu/GitHub/spacy-ios/neural-dependency-parser/data.py'>

In [4]:
config = Config()


In [ ]:
data = data.load_and_preprocess_data(
    max_batch_size=config.batch_size)

Graphs are fed into the whole pipeline pre-made through `TrainIterable` in `data.py`, via the variable `training_graphs`. 

#### prediction pathway
use minibatch_parse -> instatiates a bunch of partial-parses with sentences -> minibatch of PartialParses created -> model predicts on minibatch to generate td_vecs -> td vecs are used with parse_step to construct the parsed sentence in PartialParse.

we have to port transducer (none of the graph bits -- I THINK just pps2feats and td_vec2trans_deprel) 